In [113]:
import os
from glob import glob
import json
import pandas as pd
from tqdm import tqdm
from matplotlib import pyplot as plt
from IPython.display import Image, display
import time
import numpy as np

import cv2
from collections import defaultdict

In [114]:
scatter = pd.read_csv('/Users/minsukchang/Research/ChartDataset/scatter/scatters.csv')


def real_corr(filename):
    direction, _, corr, exp = filename.split('_')
    row = scatter[(scatter['scatter_type'] == direction) & (scatter['corr'] == int(corr)) & (scatter['exp'] == int(exp))]
    if row.shape[0] == 0:
        print(filename)
    return row['corr_computed'].values[0]

In [115]:
labels = [
    # Neutral
    ['As the usage of internet increases, so does the homicide rate in the city.', 'Increasing internet usage', 'Higher homicide rate in the city'],
    ['People who eat more cheese, tend to be better at dancing.', 'Eating more cheese', 'Better at dancing'],
    ['The more students wear glasses, the later the gym closes on campus.', 'More students wearing glasses', 'Gym closing later on campus'],
    ['A city with more lawyers, tends to have more trees.', 'Having more lawyers in a city', 'Having more trees in the city'],

    # Positive
    ['The more often students eat breakfast, the higher their GPAs are.', 'Eating breakfast more often', 'Having a higher GPA'],
    ['A worker with a longer commute, tends to be more stressed.', 'Commuting for a longer time', 'Being more stressed'],
    ['People who sleep more, tend to be happier with their lives.', 'Sleeping more', 'Being happier with life'],
    ['As the number of environmental regulations increases, so does the air quality in the city.', 'Having more environmental regulations', 'Improving air quality in the city'],

    # New Neutral
    ['The more people buy socks, the more pigeons appear in the park.', 'Buying more socks', 'Seeing more pigeons in the park'],
    ['As the number of cats in the city increases, the library\'s carpet gets replaced more often.', 'Number of cats in city', 'Library carpet replacement frequency'],

    # Semi-positive
    ['The more drivers wear seatbelts, the more survivors there are in accidents.', 'Percentage of drivers wearing seatbelts', 'More accident survivors'],
    ['The more probiotic yogurt people buy, the more toilet paper sales go up.', 'Probiotic yogurt sales', 'Toilet paper sales'],
]

In [116]:
def generate_base_components():
    return {
        "bubble": {
            "type": "react-component",
            "path": "gaze/assets/bubbleEarly.jsx",
            "response": [
                {
                    "id": "answer",
                    "prompt": "",
                    "required": True,
                    "location": "sidebar",
                    "type": "reactive"
                },
            ],
            "parameters": {
                "ratio": 0.08,
            },
            "instructionLocation": "belowStimulus",
            "nextButtonLocation": "belowStimulus"
        },
        "plain": {
            "type": "react-component",
            "path": "gaze/assets/plain.jsx",
            "response": [
                {
                    "id": "answer",
                    "prompt": "",
                    "required": True,
                    "location": "sidebar",
                    "type": "reactive"
                },
            ],
            "instructionLocation": "belowStimulus",
            "nextButtonLocation": "belowStimulus"
        },
        "draw": {
            "type": "react-component",
            "path": "gaze/assets/draw.jsx",
            "response": [
                {
                    "id": "answer",
                    "prompt": "",
                    "required": True,
                    "location": "sidebar",
                    "type": "reactive"
                },
            ],
            "instructionLocation": "belowStimulus",
            "nextButtonLocation": "belowStimulus"
        }
    }

In [117]:
def create_rei_10_scale():
    return {
        "rei_10_scale": {
            "type": "questionnaire",
            "response": [
                {
                    "id": "rei_rational_1",
                    "prompt": "I have a logical mind.",
                    "required": True,
                    "location": "aboveStimulus",
                    "type": "likert",
                    "numItems": 5,
                    "leftLabel": "Completely False",
                    "rightLabel": "Completely True",
                    "withDivider": True
                },
                {
                    "id": "rei_rational_2",
                    "prompt": "I prefer complex problems to simple problems.",
                    "required": True,
                    "location": "aboveStimulus",
                    "type": "likert",
                    "numItems": 5,
                    "leftLabel": "Completely False",
                    "rightLabel": "Completely True",
                    "withDivider": True
                },
                {
                    "id": "rei_rational_3",
                    "prompt": "I am not a very analytical thinker.",
                    "required": True,
                    "location": "aboveStimulus",
                    "type": "likert",
                    "numItems": 5,
                    "leftLabel": "Completely False",
                    "rightLabel": "Completely True",
                    "withDivider": True
                },
                {
                    "id": "rei_rational_4",
                    "prompt": "I don't reason well under pressure.",
                    "required": True,
                    "location": "aboveStimulus",
                    "type": "likert",
                    "numItems": 5,
                    "leftLabel": "Completely False",
                    "rightLabel": "Completely True",
                    "withDivider": True
                },
                {
                    "id": "rei_rational_5",
                    "prompt": "I have no problem thinking things through carefully.",
                    "required": True,
                    "location": "aboveStimulus",
                    "type": "likert",
                    "numItems": 5,
                    "leftLabel": "Completely False",
                    "rightLabel": "Completely True",
                    "withDivider": True
                },
                {
                    "id": "rei_experiential_1",
                    "prompt": "I believe in trusting my hunches.",
                    "required": True,
                    "location": "aboveStimulus",
                    "type": "likert",
                    "numItems": 5,
                    "leftLabel": "Completely False",
                    "rightLabel": "Completely True",
                    "withDivider": True
                },
                {
                    "id": "rei_experiential_2",
                    "prompt": "I trust my initial feelings about people.",
                    "required": True,
                    "location": "aboveStimulus",
                    "type": "likert",
                    "numItems": 5,
                    "leftLabel": "Completely False",
                    "rightLabel": "Completely True",
                    "withDivider": True
                },
                {
                    "id": "rei_experiential_3",
                    "prompt": "I like to rely on my intuitive impressions.",
                    "required": True,
                    "location": "aboveStimulus",
                    "type": "likert",
                    "numItems": 5,
                    "leftLabel": "Completely False",
                    "rightLabel": "Completely True",
                    "withDivider": True
                },
                {
                    "id": "rei_experiential_4",
                    "prompt": "I don't like situations in which I have to rely on intuition.",
                    "required": True,
                    "location": "aboveStimulus",
                    "type": "likert",
                    "numItems": 5,
                    "leftLabel": "Completely False",
                    "rightLabel": "Completely True",
                    "withDivider": True
                },
                {
                    "id": "rei_experiential_5",
                    "prompt": "Intuition can be a very useful way to solve problems.",
                    "required": True,
                    "location": "aboveStimulus",
                    "type": "likert",
                    "numItems": 5,
                    "leftLabel": "Completely False",
                    "rightLabel": "Completely True"
                }
            ]
        }
    }

def create_sais_7_scale():
    return {
        "sais_7_scale": {
            "type": "questionnaire",
            "response": [
                {
                    "id": "sais_1",
                    "prompt": "I feel uncomfortable when I don't know what to expect in a situation.",
                    "required": True,
                    "location": "aboveStimulus",
                    "type": "likert",
                    "numItems": 6,
                    "leftLabel": "Completely Disagree",
                    "rightLabel": "Completely Agree",
                    "withDivider": True
                },
                {
                    "id": "sais_2",
                    "prompt": "I find it difficult to make decisions when information is unclear or incomplete.",
                    "required": True,
                    "location": "aboveStimulus",
                    "type": "likert",
                    "numItems": 6,
                    "leftLabel": "Completely Disagree",
                    "rightLabel": "Completely Agree",
                    "withDivider": True
                },
                {
                    "id": "sais_3",
                    "prompt": "I prefer situations where I have all the details clearly laid out.",
                    "required": True,
                    "location": "aboveStimulus",
                    "type": "likert",
                    "numItems": 6,
                    "leftLabel": "Completely Disagree",
                    "rightLabel": "Completely Agree",
                    "withDivider": True
                },
                {
                    "id": "sais_4",
                    "prompt": "Uncertain social situations make me feel anxious.",
                    "required": True,
                    "location": "aboveStimulus",
                    "type": "likert",
                    "numItems": 6,
                    "leftLabel": "Completely Disagree",
                    "rightLabel": "Completely Agree",
                    "withDivider": True
                },
                {
                    "id": "sais_5",
                    "prompt": "I get frustrated when things are ambiguous or unclear.",
                    "required": True,
                    "location": "aboveStimulus",
                    "type": "likert",
                    "numItems": 6,
                    "leftLabel": "Completely Disagree",
                    "rightLabel": "Completely Agree",
                    "withDivider": True
                },
                {
                    "id": "sais_6",
                    "prompt": "I need clear instructions before I feel comfortable starting a task.",
                    "required": True,
                    "location": "aboveStimulus",
                    "type": "likert",
                    "numItems": 6,
                    "leftLabel": "Completely Disagree",
                    "rightLabel": "Completely Agree",
                    "withDivider": True
                },
                {
                    "id": "sais_7",
                    "prompt": "I find it stressful when I cannot predict the outcome of a situation.",
                    "required": True,
                    "location": "aboveStimulus",
                    "type": "likert",
                    "numItems": 6,
                    "leftLabel": "Completely Disagree",
                    "rightLabel": "Completely Agree"
                }
            ]
        }
    }



In [118]:
def create_nfc_scale():
    return {
        "nfc_scale": {
            "type": "questionnaire",
            "response": [
                {
                    "id": "nfc_1",
                    "prompt": "I would prefer complex to simple problems.",
                    "required": True,
                    "location": "aboveStimulus",
                    "type": "likert",
                    "numItems": 5,
                    "leftLabel": "Strongly Disagree",
                    "rightLabel": "Strongly Agree",
                    "withDivider": True
                },
                {
                    "id": "nfc_2",
                    "prompt": "I like to have the responsibility of handling a situation that requires a lot of thinking.",
                    "required": True,
                    "location": "aboveStimulus",
                    "type": "likert",
                    "numItems": 5,
                    "leftLabel": "Strongly Disagree",
                    "rightLabel": "Strongly Agree",
                    "withDivider": True
                },
                {
                    "id": "nfc_3",
                    "prompt": "Thinking is not my idea of fun.",
                    "required": True,
                    "location": "aboveStimulus",
                    "type": "likert",
                    "numItems": 5,
                    "leftLabel": "Strongly Disagree",
                    "rightLabel": "Strongly Agree",
                    "withDivider": True
                },
                {
                    "id": "nfc_4",
                    "prompt": "I would rather do something that requires little thought than something that is sure to challenge my thinking abilities.",
                    "required": True,
                    "location": "aboveStimulus",
                    "type": "likert",
                    "numItems": 5,
                    "leftLabel": "Strongly Disagree",
                    "rightLabel": "Strongly Agree",
                    "withDivider": True
                },
                {
                    "id": "nfc_5",
                    "prompt": "I really enjoy a task that involves coming up with new solutions to problems.",
                    "required": True,
                    "location": "aboveStimulus",
                    "type": "likert",
                    "numItems": 5,
                    "leftLabel": "Strongly Disagree",
                    "rightLabel": "Strongly Agree",
                    "withDivider": True
                },
                {
                    "id": "nfc_6",
                    "prompt": "I would prefer a task that is intellectual, difficult, and important to one that is somewhat important but does not require much thought.",
                    "required": True,
                    "location": "aboveStimulus",
                    "type": "likert",
                    "numItems": 5,
                    "leftLabel": "Strongly Disagree",
                    "rightLabel": "Strongly Agree"
                }
            ]
        }
    }


In [119]:
def create_default_components():
    return {
        "phase1_intro": {
            "type": "markdown",
            "path": "gaze2/assets/phase1_intro.md",
            "response": []
        },
        "phase2_intro": {
            "type": "markdown",
            "path": "gaze2/assets/phase2_intro.md",
            "response": []
        },
        # "phase3_intro": {
        #     "type": "markdown",
        #     "path": "gaze2/assets/phase3_intro.md",
        #     "response": []
        # },
        "survey_intro": {
            "type": "markdown",
            "path": "gaze2/assets/survey_intro.md",
            "response": []
        },
        "consent": {
            "type": "markdown",
            "path": "shared/consent.md",
            "nextButtonText": "I agree",
            "response": []
        },
        "introduction": {
            "type": "markdown",
            "path": "gaze2/assets/introduction.md",
            "response": []  
        },
        "example_before": {
            "type": "markdown",
            "path": "gaze2/assets/example_before.md",
            "response": []
        },
        "example_after": {
            "type": "markdown",
            "path": "gaze2/assets/example_after.md",
            "response": []
        },
        "demographics": {
            "type": "markdown",
            "path": "shared/blank.md",
            "response": [
                {
                    "id": "gender",
                    "prompt": "What is your **gender**?",
                    "required": True,
                    "location": "aboveStimulus",
                    "type": "radio",
                    "withOther": True,
                    "options": [
                        "Woman",
                        "Man",
                        "Prefer not to say"
                    ],
                    "withDivider": True

                },
                {
                    "id": "age",
                    "prompt": "What is your **age**?",
                    "required": True,
                    "location": "aboveStimulus",
                    "type": "radio",
                    "options": [
                        "Under 18 years",
                        "18-24 years",
                        "25-34 years",
                        "35-44 years",
                        "45-54 years",
                        "55-64 years",
                        "65 years or older",
                        "Prefer not to say"
                    ],
                    "withDivider": True
                },
                {
                    "id": "education",
                    "prompt": "What is the **highest degree or level of education** you have completed?",
                    "required": True,
                    "location": "aboveStimulus",
                    "type": "radio",
                    "withOther": True,
                    "options": [
                        "Less than high school",
                        "High school diploma or equivalent",
                        "Bachelor's degree or equivalent",
                        "Master's degree or equivalent",
                        "Doctoral degree or equivalent"
                    ],
                    "withDivider": True
                }
            ]
        },
    }



def create_phase1_components():
    components = {}
    for corr in [1, 3, 5, 7]:
        for i in range(1, 3): # for phase 1
            for direction in ["pos", "neg"]:
                components[f"plain_{corr}_{i}_{direction}"] = {
                    "baseComponent": "plain",
                    "parameters": {
                        "image": f"https://raw.githubusercontent.com/jangsus1/ChartQA/main/scatter/{direction}_scatter_{corr}_{i}.png",
                        "example": False,
                        "correlation": real_corr(f"{direction}_scatter_{corr}_{i}"),
                        "seconds": 5,
                    }
                }
    return components


def create_phase2_components():
    components = defaultdict(dict)
    for corr in [1, 3, 5, 7]:
        for i in range(6, 8):  # for phase 2
            for direction in ["pos"]:
                for label_idx, label in enumerate(labels):
                    label, x, y = label
                    components[label][f"phase2_{corr}_{i}_{direction}_{label_idx}"] = {
                        "baseComponent": "bubble",
                        "parameters": {
                            "image": f"https://raw.githubusercontent.com/jangsus1/ChartQA/main/scatter/{direction}_scatter_{corr}_{i}.png",
                            "example": False,
                            "correlation": real_corr(f"{direction}_scatter_{corr}_{i}"),
                            "label": label,
                            "X": x,
                            "Y": y,
                            "corr": corr,
                            "exp": i,
                            "direction": direction,
                            "revealSeconds": 3,
                            "seconds": 10,
                        }
                    }
    components[label][f"example_1"] = {
        "baseComponent": "bubble",
        "parameters": {
            "image": f"https://raw.githubusercontent.com/jangsus1/ChartQA/main/scatter/pos_scatter_3_2.png",
            "example": False,
            "correlation": real_corr("pos_scatter_3_2"),
            "label": "The more hours people spend exercising, the better their cardiovascular health becomes.",
            "X": "Hours spent exercising",
            "Y": "Cardiovascular health score",
            "corr": 3,
            "exp": 2,
            "direction": "pos",
            "revealSeconds": 3,
            "seconds": 10,
        }
    }
    components[label][f"example_2"] = {
        "baseComponent": "bubble",
        "parameters": {
            "image": f"https://raw.githubusercontent.com/jangsus1/ChartQA/main/scatter/pos_scatter_5_1.png",
            "example": False,
            "correlation": real_corr("pos_scatter_5_1"),
            "label": "Cities with more public parks tend to have higher rates of outdoor exercise.",
            "X": "Number of public parks in city",
            "Y": "Rate of outdoor exercise",
            "corr": 5,
            "exp": 1,
            "direction": "pos",
            "revealSeconds": 3,
            "seconds": 10,
        }
    }
    return components

# def create_phase3_components():
#     components = {}
#     for label_idx, label in enumerate(labels):
#         label, x, y = label
#         components[f"phase3_{label_idx}"] = {
#             "baseComponent": "draw",
#             "parameters": {
#                 "image": "https://raw.githubusercontent.com/jangsus1/ChartQA/main/scatter/empty_scatter.png",
#                 "label": label,
#                 "X": x,
#                 "Y": y,
#             }
#         }
#     return components

def sequence_generator(phase1_components, phase2_components):
    groups = [{
        "id": label[:30],
        "order": "latinSquare",
        "numSamples": 1,
        "components": list(phase2_components[label].keys())
    } for label in phase2_components.keys()]
    
    sequence = {
        "order": "fixed",
        "components": [
            "consent",
            "demographics",
            "introduction",
            "phase1_intro",
            {
                "id": "phase1",
                "order": "latinSquare",
                "components": list(phase1_components.keys())
            },
            "phase2_intro",
            "example_before",
            "example_1",
            "example_2",
            "example_after",
            {
                "id": "phase2",
                "order": "latinSquare",
                "components": groups
            },
            "survey_intro",
            "nfc_scale",
            "rei_10_scale",
            "sais_7_scale",
        ]
    }
    return sequence

In [120]:
default_components = create_default_components()
nfc_scale = create_nfc_scale()
# tipi_scale = create_tipi_scale()
rei_10_scale = create_rei_10_scale()
sais_7_scale = create_sais_7_scale()
phase1_components = create_phase1_components()
phase2_components = create_phase2_components()
# phase3_components = create_phase3_components()
components = default_components | phase1_components | nfc_scale | rei_10_scale | sais_7_scale
for component in phase2_components.values():
    components |= component

sequence = sequence_generator(phase1_components, phase2_components)
baseComponents = generate_base_components()
print(f"Total number of components: {len(components)}")

Total number of components: 125


In [121]:
prolificRedirection = "https://app.prolific.com/submissions/complete?cc=C12B2Y1Z"

In [122]:
with open("config.json", "r") as f:
    config = json.load(f)
config['uiConfig']['studyEndMsg'] = f"**Thank you for completing the study. You may click this link and return to Prolific**: [{prolificRedirection}]({prolificRedirection})"
config['components'] = components
config['sequence'] = sequence
config['baseComponents'] = baseComponents
with open("config.json", "w") as f:
    json.dump(config, f, indent=4)